In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split, cross_val_score

In [2]:
from keras.preprocessing.text import text_to_word_sequence

stop_words = pd.read_csv('../data/stopwords.csv')['words'].values

def remove_stop_words(text):
    word_tokens = text_to_word_sequence(text) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    return ' '.join(filtered_sentence)

print(stop_words)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


['only' 'y' 'by' 'am' 'most' 'me' 'same' 'these' 'so' 'some' 'why' 'down'
 'had' 'd' 'at' 'having' 'those' 'has' 'few' 'theirs' "you've" 'more' 'i'
 'than' 'through' 'be' 'what' 'where' 'myself' 'which' 'doing' 'ours'
 'will' 'in' 'both' 'do' 'it' 'o' 'on' 'yours' 'once' 'ourselves' 'here'
 'about' "it's" 'my' 'for' 'her' 'then' 'after' "should've" 'from' 'each'
 'when' 'does' 'now' 'off' 'don' 'are' 'we' 'itself' 'should' 'his'
 'between' 'our' 'were' 'under' 'other' 'all' 'she' 'won' 'been' "you're"
 'how' 'did' 'yourself' 'they' 'into' 'there' 've' 'such' 't' 's' 'and'
 'over' 'to' 'just' 'was' 'being' 'because' 'if' 'who' 'further' 'the'
 'any' "that'll" 'themselves' 'as' 'again' "you'd" 'until' 'he' 'him'
 'this' 'or' 'of' 'below' 'an' "she's" 'weren' 'm' 'their' 'ma' 'up' 'll'
 'whom' 'hers' 'can' 'you' 'them' 'very' 'a' 'herself' 'before' 'too'
 'himself' 'during' 're' 'out' 'its' 'above' 'own' 'have' 'while'
 'yourselves' 'that' 'with' "you'll" 'is' 'your']


In [3]:
remove_stop_words("testing to see if this will not works hooray")

'testing see not works hooray'

In [4]:
data = pd.read_csv('../data/twitter-airline-sentiment.csv')

In [5]:
data['text'] = list(map(remove_stop_words, data['text'].values))
data.head()

,text,pos,neg
0,said,0.0,0.0
1,plus youve added commercials experience tacky,1.0,0.0
2,didnt today must mean need take another trip,0.0,0.0
3,really aggressive blast obnoxious entertainmen...,0.0,1.0
4,really big bad thing,0.0,1.0


In [6]:
# classifier = GaussianNB()
classifier = RandomForestClassifier()
# vectorizer = TfidfVectorizer()
vectorizer = CountVectorizer()

In [7]:
train_x, test_x, train_y, test_y = train_test_split(
            np.array(data.text), 
            np.array(data.pos),
            test_size=0.2, 
            random_state=3945)

In [8]:
tfidf_train_x = vectorizer.fit_transform(train_x)
classifier.fit(tfidf_train_x.toarray(), train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [9]:
tfidf_test_x = vectorizer.transform(test_x)
scores = cross_val_score(classifier, tfidf_test_x.toarray(), test_y, cv=5)
acc = scores.mean()
print("Accuracy: %0.2f percent" % (acc *100))

Accuracy: 85.73 percent


In [10]:
mess = [
    'woah that', 
    'fuck that',
    'i think it was fine but that shit was horrible!',
    'meg how are you?',
    'i really did not like this!',
    'i hated this',
    "that's not really nice",
    "i'm not into you",
    "i want to try things out",
    "i really think this will work out great!",
    "this was great!",
    "this was not great!",
    "i didn't get how it worked? what is wrong?",
    "your service is subpar",
    "service",
    "good",
    "bad",    
    "hate",
    "love",

]
t = vectorizer.transform(list(map(remove_stop_words, mess))).toarray();
output = classifier.predict(t)
proba = classifier.predict_proba(t)

for i ,m in enumerate(mess):
    print(m, ' == ', output[i], ' hmm ', proba[i])

woah that  ==  0.0  hmm  [0.88958668 0.11041332]
fuck that  ==  0.0  hmm  [0.88958668 0.11041332]
i think it was fine but that shit was horrible!  ==  0.0  hmm  [1. 0.]
meg how are you?  ==  0.0  hmm  [0.88958668 0.11041332]
i really did not like this!  ==  0.0  hmm  [1. 0.]
i hated this  ==  0.0  hmm  [0.88958668 0.11041332]
that's not really nice  ==  0.0  hmm  [0.6 0.4]
i'm not into you  ==  0.0  hmm  [1. 0.]
i want to try things out  ==  0.0  hmm  [0.88833333 0.11166667]
i really think this will work out great!  ==  0.0  hmm  [0.74047619 0.25952381]
this was great!  ==  0.0  hmm  [0.74285714 0.25714286]
this was not great!  ==  1.0  hmm  [0.40952381 0.59047619]
i didn't get how it worked? what is wrong?  ==  1.0  hmm  [0.4 0.6]
your service is subpar  ==  0.0  hmm  [0.99285714 0.00714286]
service  ==  0.0  hmm  [0.99285714 0.00714286]
good  ==  0.0  hmm  [0.9 0.1]
bad  ==  0.0  hmm  [1. 0.]
hate  ==  0.0  hmm  [0.98297872 0.01702128]
love  ==  1.0  hmm  [0.025 0.975]


In [33]:
from IPython.display import clear_output
import os

import urllib, json
import urllib.request

from dotenv import load_dotenv
load_dotenv()

TLGRM_SECRET = os.getenv('TLGRM_SECRET')
TLGRM_MIKE = os.getenv('TLGRM_MIKE')

def telegram_call(method, query = {}):
    url = 'http://api.telegram.org/bot{}/{}?{}'.format(TLGRM_SECRET, method, urllib.parse.urlencode(query))
    response = urllib.request.urlopen(url)
    return json.loads(response.read().decode("utf-8"))

def telegram_bot(respond):
    last_offset = 0
    
    while True:
        data = telegram_call('getupdates', {'offset': last_offset})
        for item in data['result']:
            last_offset = item['update_id'] + 1
            if 'message' in item:
                if 'text' in item['message']:
                    text = item['message']['text']
                    chat_id = item['message']['chat']['id']
                    response = respond(text)
                    telegram_call('sendmessage', {
                        'chat_id': chat_id,
                        'text': response
                    })

                    print('FROM: {}\nSAYS: {}\nRESPONSE: {}\n\n'.format(chat_id, text, response))

In [ ]:
def respond(text):
    mess = [text]
    t = vectorizer.transform(list(map(remove_stop_words, mess))).toarray();
    output = classifier.predict(t)
    proba = classifier.predict_proba(t)
    
    return '{}'.format(output[0])

telegram_bot(respond)

FROM: -221717641
SAYS: i think this will work find
RESPONSE: 0.0


FROM: -221717641
SAYS: fine
RESPONSE: 0.0


FROM: -221717641
SAYS: exceptional service
RESPONSE: 1.0


FROM: -221717641
SAYS: this is too slow
RESPONSE: 0.0


FROM: -221717641
SAYS: i am happy about this service
RESPONSE: 0.0


